In [1]:
from dotenv import load_dotenv
import pandas as pd
import json
import os

In [2]:
#For now, load in the whole df and then filter it down to the relevant rows
def get_data(fname):
    with open("./datasets/fa.json", "r", encoding="utf-8") as f:
        #json.dump(data, f, indent=4, ensure_ascii=False)
        data = json.load(f)

In [3]:
# Load environment variables from .env file
load_dotenv()

# Access the API key
llama_api_key = os.getenv("LLAMA_API_KEY")

In [4]:
print(llama_api_key)

LL-kklYmOKE4pxJNTBTKjOIwy5uum8wrkni2bCUuJ3m4ExNEJI2Vdq07buAFj8uRwcO


In [5]:
from llamaapi import LlamaAPI

In [6]:
llm = LlamaAPI(llama_api_key)

In [7]:
# API Request JSON Cell
api_request_json = {
  "model": "llama-13b-chat",
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."},
    {"role": "user", "content": "Hi, happy llama day!"},
  ]
}

# Make your request and handle the response
response = llm.run(api_request_json)
print(json.dumps(response.json(), indent=2))

{
  "created": 1711672374,
  "model": "llama-13b-chat",
  "usage": {
    "prompt_tokens": 24,
    "completion_tokens": 37,
    "total_tokens": 61
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "LL-HELLO THERE, LL-HAPPY LL-LAMA DAY TO YOU LL-TOO! LL-HOW LL-MAY I LL-HELP YOU LL-TODAY?",
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ]
}


In [8]:
api_request_json_farsi = {
  "model": "llama-13b-chat",
  "messages": [
    {"role": "system", "content": "You are a Farsi chatbot. Respond to the Farsi query with a Farsi response."},
    {"role": "user", "content": "سلام. چطوری؟"},
  ]
}

# Make your request and handle the response
response = llm.run(api_request_json_farsi)
print(json.dumps(response.json(), indent=2))


{
  "created": 1711672557,
  "model": "llama-13b-chat",
  "usage": {
    "prompt_tokens": 32,
    "completion_tokens": 43,
    "total_tokens": 75
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u0633\u0644\u0627\u0645! \u0686\u062c\u0648\u0631 \u0627\u0633\u062a\u06cc! \u0645\u06cc\u200c\u062e\u0648\u0631\u0645 \u06a9\u0647 \u0639\u0632\u06cc\u0632\u0627\u0646\u06cc \u0628\u0627\u0634\u06cc\u0645. \u06a9\u062c\u0627 \u0645\u06cc\u200c\u06af\u0648\u06cc\u06cc\u062f\u061f",
        "function_call": null
      },
      "finish_reason": "max_token"
    }
  ]
}


In [9]:
decoded_response = response.json()
farsi_response = decoded_response['choices'][0]['message']['content']

In [10]:
print(farsi_response)

سلام! چجور استی! می‌خورم که عزیزانی باشیم. کجا می‌گویید؟


In [7]:
# Build the API request
api_request_json = {
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "days": {
                        "type": "number",
                        "description": "for how many days ahead you wants the forecast",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            },
            "required": ["location", "days"],
        }
    ],
    "stream": False,
    "function_call": "get_current_weather",
}

In [8]:
# Build the API request
api_request_json_farsi = {
    "messages": [
        {"role": "user", "content": "سلام. چطوری"},
    ],
    "functions": [
        {
            "name": "farsi_llm_chatbot",
            "description": "Respond to the Farsi query in Farsi",
            # "parameters": {
            #     "type": "object",
            #     "properties": {
            #         "location": {
            #             "type": "string",
            #             "description": "The city and state, e.g. San Francisco, CA",
            #         },
            #         "days": {
            #             "type": "number",
            #             "description": "for how many days ahead you wants the forecast",
            #         },
            #         "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            # #     },
            # },
            # "required": ["location", "days"],
        }
    ],
    "stream": False,
    # "function_call": "get_current_weather",
}

In [9]:
response = llm.run(api_request_json)